In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pandas as pd

In [4]:
import os
os.chdir('/content/drive')

In [5]:
stopwords=open("/content/drive/MyDrive/한국어불용어100_.txt",'r',encoding='UTF8')
stop=stopwords.read()
stop=stop.split()

In [6]:
# 불용어 처리 함수
def stopword(word_tokenize):
  return [w for w in word_tokenize if w not in stop_list]

In [7]:
!pip install konlpy

     |████████████████████████████████| 19.4 MB 1.3 MB/s 
     |████████████████████████████████| 86 kB 5.9 MB/s 
     |████████████████████████████████| 448 kB 52.6 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [8]:
!pip install collection

  Created wheel for collection: filename=collection-0.1.6-py3-none-any.whl size=5117 sha256=d2f2669047a4345a49e097aa66395178840d9ecce82d4ab5692f8b825afe1352
  Stored in directory: /root/.cache/pip/wheels/45/ed/3c/29a1c0600bb908a2e7421dc3a3775cbaed434a71b4ed93a439
Successfully built collection


In [9]:
!pip install matplotlib

In [36]:
from konlpy.tag import Okt
from collections import Counter
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import pandas as pd
from time import time
from concurrent.futures import ProcessPoolExecutor


In [69]:
# 품사태깅 함수 
# 인풋 : 데이터 프레임 형태로 행별로 글들이 담겨있는 형태

# 순차처리

def part_tagging(data):
  from konlpy.tag import Okt

  Okt = Okt()
  morphs=[Okt.pos(sentence) for sentence in data]

  return morphs # 품사 태깅해서 하나의 리스트형태로 반환

## pandas에서 병렬처리 적용

pandas는 내부적으로 병렬처리 기능을 지원하지 않기 때문에 데이터가 크고 전처리 함수가 복잡하면 시간이 매우 오래 걸릴 수 있습니다. 

- map과 apply만으로도 충분히 빠른 성능을 낼 수 있기 때문에 multiprocessing이 항상 해답은 아님

In [83]:
# 품사태깅 함수 
# 인풋 : 데이터 프레임 형태로 행별로 글들이 담겨있는 형태

# 병렬처리 적용

def tagging(data):
  from konlpy.tag import Okt
  Okt = Okt()
  return Okt.pos(data)

def parallel_processing_part_tagging(data):
  
  pool = ProcessPoolExecutor(max_workers=2)
  morphs = list(pool.map( tagging,[sentence for sentence in data]))

  return morphs # 품사 태깅해서 하나의 리스트형태로 반환

In [34]:
sample = pd.DataFrame({'contents':['시작']})

python_news =["파이썬 런타임을 변경해 멀티스레드 성능을 강화하는 새로운 프로젝트가 제안됐다. 파이썬 개발팀도 해당 개념 증명에 관심을 보였다. 파이썬의 고질적인 약점(멀티스레드 환경에서 잘 확장되지 않는다는 것)이 해당 프로그래밍 언어 개발팀의 새로운 목표가 될지도 모른다. 지난 10월 7일(현지 시각) 개발자 샘 그로스가 파이썬의 참조 구현 C파이썬의 핵심 구성요소 ‘GIL(Global Interpreter Lock)’에 관한 변경사항을 제안했다.만약 파이썬 개발팀에서 그의 제안을 받아들인다면 이는 파이썬이 런타임에서 멀티스레드로부터 객체 액세스를 직렬화하는 방식을 변경하고, 멀티스레드 성능을 크게 향상시킬 것으로 예상된다. GIL은 오랫동안 C파이썬(그리고 파이썬)에서 더 나은 멀티스레드 성능의 장애물로 간주돼 왔다. 수년 동안 이를 제거하려는 많은 시도가 있었다. 하지만 대다수의 기존 파이썬 애플리케이션을 느리게 만들어 단일 스레드 성능을 저하시키는 문제를 해결할 수 없었다.스레드 및 멀티프로세싱을 다루는 파이썬의 현 메타포는 높은 병렬처리가 불가능한 건 아니다. 하지만 개발자가 작업을 완료하기 위해서는 다스크(Dask) 등의 서드파티 모듈을 사용해야 했다.새로운 제안은 파이썬 객체의 참조 카운팅 방식을 변경하여 객체를 소유한 스레드의 참조가 다른 스레드의 참조와 다르게 처리되도록 한다. 그로스에 따르면 단일 스레드 성능 벤치마크에서 GIL를 제거한 새로운 인터프리터는 C파이썬 3.9 인터프리터보다 약 10% 빠른 것으로 나타났다. 또한 일부 벤치마크에서 멀티스레드 성능은 새로운 스레드에 따라 거의 선형적으로 확장됐다. 예를 들면 20개의 스레드를 사용할 때 한 벤치마크에서는 최대 18.1배, 다른 벤치마크에서는 최대 19.8배 빨라졌다고 그는 언급했다.이어서 이는 파이썬의 내부(예: C파이썬)에서 작동하는 기존 파이썬 라이브러리를 상당 수 다시 작성해야 할 만큼 큰 변경사항이라고 그로스는 덧붙였다.파이썬(Python)과 자바(Java)는 가장 많이 사용되는, 인기 있는 프로그래밍 언어 중 하나이다. 올해 10월 티오베 지수(TIOBE Index)에서 파이썬이 자바를 제치며 20여 년 만에 가장 인기 있는 프로그래밍 언어로 올라섰다. 자바와 파이썬 모두 객체 지향 프로그래밍 언어로 오늘날 데이터 과학에서 가장 선호되는 언어이기도 하다. 그런데 데이터 과학을 위해선 어떤 언어를 선택해야 할까? 애널리티스 인디아 매거진이 데이터 과학 관점에서 파이썬과 자바의 문법, 퍼포먼스, 프레임워크 및 도구를 비교했다.자바와 파이썬의 주요 차이점 중 하나는 문법에 있다. 프로그래머는 자바에서 코드를 작성할 때 변수의 데이터 유형을 정의해야 한다. 그리고 이 데이터 유형은 명시적으로 변경할 수 없으며 프로그램의 수명 동안 유지된다. 이는 자바가 강력한 타입화된 언어로 만든다. 반면 파이썬의 경우 변수의 데이터 타입이 런타임에 자동으로 정의된다. 변수의 타입은 프로그램 수명 동안 변경될 수 있으며, 파이썬은 동적으로 타입화된 프로그래밍 언어가 된다. 동적 타이핑을 사용하면 해당 언어를 사용하기 쉽고 코드 줄 수를 줄일 수 있다. 또한 자바에서는 쌍반점(;)을 사용해 구문을 끝내지 않거나 중괄호를 생략하면 컴파일 중 오류가 발생한다. 반면 파이썬은 이러한 복잡한 프로그래밍 구조를 따르지 않아 학습과 사용이 더 쉽다. 속도의 측면에서는 자바가 더 유리하다. 파이썬은 소스 코드를 한 줄 한 줄 읽는 해석된 언어이다. 이는 파이썬의 소스 코드 속도를 느리게 만든다. 실제로 파이썬 프로그래밍에서 디버깅은 런타입 동안 발생한다. 반면 자바는 동시에 여러 개의 계산을 수행한다. 파이썬과 자바 모두 데이터 과학, 데이터 분석 및 머신러닝 작업을 지원하는 라이브러리 목록을 제공한다. 판다스(Pandas) : 판다스는 파이썬에서 가장 인기 있는 오픈 소스 라이브러리이다. 대용량 데이터 집합을 처리하는 데 사용되며, 데이터 정렬, 고급 인덱싱 및 누락된 데이터 처리와 같은 직관적인 기능과 함께 유연하고 빠르고 표현력이 뛰어난 데이터 구조를 제공한다. 싸이파이(SciPy, Scientific Python) : 이름에서 알 수 있듯이 싸이파이는 과학, 복합적 수학, 공학과 관련된 문제를 푸는 데 사용된다. 통계, 선형 대수, 최적화 및 통합을 위한 루틴을 제공한다. 넘파이(NumPy, Numerical Python) : 넘파이는 통계 및 수학적 계산을 위한 기초적인 도구이다. 넘파이 위에 싸이파이, 판다스, 매트플롯 라이브러리(Matplotlib), 스테츠모델스(Statsmodels)를 포함한 라이브러리들이 세워진다. 텐서플로(TensoFlow) : 텐서플로는 구글 브레인 팀이 개발한 오픈 소스 라이브러리로 주로 파이썬의 딥 러닝 애플리케이션에 사용된다. ML 기반 애플리케이션의 배치를 돕는다. WEKA 3: 지식 분석을 위한 와이카토(Waikato) 환경의 줄임말로 데이터 구현과 처리 도구를 제공하는 오픈 소스 소프트웨어이다. 예측 모델링, 데이터 마이닝 및 분석에 주로 사용한다. 아파치 스파크(Apache Spark) : 빅데이터 처리를 위한 사용하기 쉽고 빠른 엔진이다. 아파치 하둡 맵리듀스(Apache Hadoop MapReduce)를 기반으로 하며 주로 대규모 데이터셋을 처리하는 데 사용된다. 여기에 스파크 SQL, 스파크 스트리밍, 스파크 머신러닝 라이브러리 등의 모듈이 내장되어있다. 자바 머신러닝(Java Machine Learning): 자바 머신러닝 라이브러리는 데이터 분류, 처리 및 클러스팅에 사용할 수 있는 방대한 머신러닝 및 데이터 마이닝 알고리즘을 제공한다. 딥러닝4j(Deeplearning4j): 딥러닝4j는 자바 프로그래머들이 머신러닝 애플리케이션을 만들 수 있도록 도와주는 오픈 소스 라이브러리이다. 파이썬의 학습 곡선이 자바만큼 가파르지 않기 때문에, 머신러닝 프로그래머, 특히 초보자들은 자바보다 파이썬을 선호한다. 실제로 파이썬은 '초보자 언어'로 꼽히는데, 머신러닝과 데이터 사이언스 온라인 학습 강좌 대부분이 초보자 친화적 기능을 위해 파이썬을 강조하고 있어 데이터 과학 분야에서 파이썬의 인기가 높아지고 있다. 한편 연구자들은 자신만의 라이브러리를 만들 때 깃허브(GitHub)와 같은 오픈 소스 플랫폼에 업로드한다. 이러한 대규모 개발자 컴퓨니티 지원으로 머신러닝 애플리케이션을 개발할 때 파이썬을 사용하는 것이 더 용이해진다."]
python_news_split=python_news[0].split('.')
python_news_split

# 출처 : https://www.ciokorea.com/news/210764
# 출처 : https://www.codingworldnews.com/news/articleView.html?idxno=6567



['파이썬 런타임을 변경해 멀티스레드 성능을 강화하는 새로운 프로젝트가 제안됐다',
 ' 파이썬 개발팀도 해당 개념 증명에 관심을 보였다',
 ' 파이썬의 고질적인 약점(멀티스레드 환경에서 잘 확장되지 않는다는 것)이 해당 프로그래밍 언어 개발팀의 새로운 목표가 될지도 모른다',
 ' 지난 10월 7일(현지 시각) 개발자 샘 그로스가 파이썬의 참조 구현 C파이썬의 핵심 구성요소 ‘GIL(Global Interpreter Lock)’에 관한 변경사항을 제안했다',
 '만약 파이썬 개발팀에서 그의 제안을 받아들인다면 이는 파이썬이 런타임에서 멀티스레드로부터 객체 액세스를 직렬화하는 방식을 변경하고, 멀티스레드 성능을 크게 향상시킬 것으로 예상된다',
 ' GIL은 오랫동안 C파이썬(그리고 파이썬)에서 더 나은 멀티스레드 성능의 장애물로 간주돼 왔다',
 ' 수년 동안 이를 제거하려는 많은 시도가 있었다',
 ' 하지만 대다수의 기존 파이썬 애플리케이션을 느리게 만들어 단일 스레드 성능을 저하시키는 문제를 해결할 수 없었다',
 '스레드 및 멀티프로세싱을 다루는 파이썬의 현 메타포는 높은 병렬처리가 불가능한 건 아니다',
 ' 하지만 개발자가 작업을 완료하기 위해서는 다스크(Dask) 등의 서드파티 모듈을 사용해야 했다',
 '새로운 제안은 파이썬 객체의 참조 카운팅 방식을 변경하여 객체를 소유한 스레드의 참조가 다른 스레드의 참조와 다르게 처리되도록 한다',
 ' 그로스에 따르면 단일 스레드 성능 벤치마크에서 GIL를 제거한 새로운 인터프리터는 C파이썬 3',
 '9 인터프리터보다 약 10% 빠른 것으로 나타났다',
 ' 또한 일부 벤치마크에서 멀티스레드 성능은 새로운 스레드에 따라 거의 선형적으로 확장됐다',
 ' 예를 들면 20개의 스레드를 사용할 때 한 벤치마크에서는 최대 18',
 '1배, 다른 벤치마크에서는 최대 19',
 '8배 빨라졌다고 그는 언급했다',
 '이어서 이는 파이썬의 내부(예: C파이썬)에서 작동하는 기존 파이썬 라이브러리를 상당 수 다시

In [35]:
for i in range(len(python_news_split)):
  sample.loc[i] = python_news_split[i]
sample

,contents
0,파이썬 런타임을 변경해 멀티스레드 성능을 강화하는 새로운 프로젝트가 제안됐다
1,파이썬 개발팀도 해당 개념 증명에 관심을 보였다
2,파이썬의 고질적인 약점(멀티스레드 환경에서 잘 확장되지 않는다는 것)이 해당 프로...
3,지난 10월 7일(현지 시각) 개발자 샘 그로스가 파이썬의 참조 구현 C파이썬의 ...
4,만약 파이썬 개발팀에서 그의 제안을 받아들인다면 이는 파이썬이 런타임에서 멀티스레드...
5,GIL은 오랫동안 C파이썬(그리고 파이썬)에서 더 나은 멀티스레드 성능의 장애물로...
6,수년 동안 이를 제거하려는 많은 시도가 있었다
7,하지만 대다수의 기존 파이썬 애플리케이션을 느리게 만들어 단일 스레드 성능을 저하...
8,스레드 및 멀티프로세싱을 다루는 파이썬의 현 메타포는 높은 병렬처리가 불가능한 건 아니다
9,하지만 개발자가 작업을 완료하기 위해서는 다스크(Dask) 등의 서드파티 모듈을 ...


# sample 데이터셋으로 순차처리랑  병렬처리를 비교해보자



In [70]:
# 순차처리 실행시간

start = time()
print(part_tagging(sample['contents']))
end = time()
print("time :", end - start)


[[('파이썬', 'Noun'), ('런타임', 'Noun'), ('을', 'Josa'), ('변경', 'Noun'), ('해', 'Verb'), ('멀티스레드', 'Noun'), ('성능', 'Noun'), ('을', 'Josa'), ('강화하는', 'Adjective'), ('새로운', 'Adjective'), ('프로젝트', 'Noun'), ('가', 'Josa'), ('제안', 'Noun'), ('됐다', 'Verb')], [('파이썬', 'Noun'), ('개발', 'Noun'), ('팀', 'Noun'), ('도', 'Josa'), ('해당', 'Noun'), ('개념', 'Noun'), ('증명', 'Noun'), ('에', 'Josa'), ('관심', 'Noun'), ('을', 'Josa'), ('보였다', 'Verb')], [('파이썬', 'Noun'), ('의', 'Josa'), ('고질', 'Noun'), ('적', 'Suffix'), ('인', 'Josa'), ('약점', 'Noun'), ('(', 'Punctuation'), ('멀티스레드', 'Noun'), ('환경', 'Noun'), ('에서', 'Josa'), ('잘', 'Verb'), ('확장', 'Noun'), ('되지', 'Verb'), ('않는다는', 'Verb'), ('것', 'Noun'), (')', 'Punctuation'), ('이', 'Noun'), ('해당', 'Noun'), ('프로그래밍', 'Noun'), ('언어', 'Noun'), ('개발', 'Noun'), ('팀', 'Noun'), ('의', 'Josa'), ('새로운', 'Adjective'), ('목표', 'Noun'), ('가', 'Josa'), ('될지도', 'Verb'), ('모른다', 'Verb')], [('지난', 'Noun'), ('10월', 'Number'), ('7일', 'Number'), ('(', 'Punctuation'), ('현지', 'Noun'), ('시각', 'Noun'), (

In [85]:
# 병렬처리 실행시간

start = time()
print(parallel_processing_part_tagging(sample['contents']))
end = time()
print("time :", end - start)

[[('파이썬', 'Noun'), ('런타임', 'Noun'), ('을', 'Josa'), ('변경', 'Noun'), ('해', 'Verb'), ('멀티스레드', 'Noun'), ('성능', 'Noun'), ('을', 'Josa'), ('강화하는', 'Adjective'), ('새로운', 'Adjective'), ('프로젝트', 'Noun'), ('가', 'Josa'), ('제안', 'Noun'), ('됐다', 'Verb')], [('파이썬', 'Noun'), ('개발', 'Noun'), ('팀', 'Noun'), ('도', 'Josa'), ('해당', 'Noun'), ('개념', 'Noun'), ('증명', 'Noun'), ('에', 'Josa'), ('관심', 'Noun'), ('을', 'Josa'), ('보였다', 'Verb')], [('파이썬', 'Noun'), ('의', 'Josa'), ('고질', 'Noun'), ('적', 'Suffix'), ('인', 'Josa'), ('약점', 'Noun'), ('(', 'Punctuation'), ('멀티스레드', 'Noun'), ('환경', 'Noun'), ('에서', 'Josa'), ('잘', 'Verb'), ('확장', 'Noun'), ('되지', 'Verb'), ('않는다는', 'Verb'), ('것', 'Noun'), (')', 'Punctuation'), ('이', 'Noun'), ('해당', 'Noun'), ('프로그래밍', 'Noun'), ('언어', 'Noun'), ('개발', 'Noun'), ('팀', 'Noun'), ('의', 'Josa'), ('새로운', 'Adjective'), ('목표', 'Noun'), ('가', 'Josa'), ('될지도', 'Verb'), ('모른다', 'Verb')], [('지난', 'Noun'), ('10월', 'Number'), ('7일', 'Number'), ('(', 'Punctuation'), ('현지', 'Noun'), ('시각', 'Noun'), (

# 병렬처리가 더 짧은 모습을 보여준다!

컴퓨터 사양이 더 좋아 코어수가 늘어나면 병렬처리 했을떄 더 짧은 수행시간을 기대할 수 있을 것 같음